In [ ]:
!pip install dateparser

In [ ]:
!pip install tqdm
from tqdm import tqdm

In [ ]:
!pip install pyts

In [ ]:
import os
import boto3
import pandas as pd
import sys
from datetime import datetime
from datetime import timedelta 
import dateparser as dp
from matplotlib import pyplot
import numpy as np
import matplotlib.pyplot as plt
from pyts.image import GramianAngularField
from mpl_toolkits.axes_grid1 import ImageGrid
from pyts.image import MarkovTransitionField
from pyts.image import RecurrencePlot
import io
import time
from keras import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten,Conv2D, MaxPooling2D
from keras.utils import np_utils
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import classification_report
from statistics import mean
import seaborn as sns
from keras.models import load_model

%matplotlib inline 

In [ ]:
BUCKET_NAME = <s3_bucket_name>
DATA_POINTS_PER_WINDOW = 21
s3Res = boto3.resource('s3')
bucket = s3Res.Bucket(BUCKET_NAME)
labelledDataCommonPath = <path_to_download_dir_on_s3>
tempDiskSaveLoc= <location_on_disk.png>
s3Client = boto3.client('s3')
INPUT_MATRIX_WIDTH = 21
ENCODED_FEATURES=2

In [ ]:
def getMTFMatrix(df, feature, index, bins=10, span=10):
    X = [df.loc[(index - timedelta(minutes=span)) : (index + timedelta(minutes=span)), feature]]
    if len(X[0]) != DATA_POINTS_PER_WINDOW:
        print("MTF error..{} Length != {}, {} point={}".format(feature, DATA_POINTS_PER_WINDOW, len(X[0]), index))
        raise Exception('MTF Length != %d, %d' %(DATA_POINTS_PER_WINDOW, len(X[0]))) 
    
    mtf = MarkovTransitionField(n_bins=bins, strategy='uniform', overlapping=False)
    x_mtf = mtf.fit_transform(X)
    return x_mtf

In [ ]:
my_bucket = s3Res.Bucket(BUCKET_NAME)

csvFileList = []

for my_bucket_object in my_bucket.objects.filter(Prefix=labelledDataCommonPath):
    if '.csv' in my_bucket_object.key:
        print(my_bucket_object.key)
        csvFileList.append(my_bucket_object.key)

In [ ]:
modelSaveLocOnDisk = <location_to_h5_model_file>

mtfCnn3 = load_model(modelSaveLocOnDisk)
mtfCnn3.summary()

# Training main loop

In [ ]:
fileIndex = 0

encodedFeatures = ['Price', 'Volume']
encoded_feature_count = len(encodedFeatures)
minVicinity = 20

NUMBER_OF_FILES_USEDTO_TRAIN = 20

predictionDf = pd.DataFrame(columns = ['CoinName', 'PredictedLabel', 'Label'])

for file_name in csvFileList[NUMBER_OF_FILES_USEDTO_TRAIN:23]:
    print(file_name)
    
    coin_name = file_name.split('/')[-1].split('_')[1]
    fileIndex +=1
    obj = s3Client.get_object(Bucket = BUCKET_NAME, Key = file_name)
    df = pd.read_csv(obj['Body'], index_col='0', parse_dates=True)
    mismatches = []
    
    df['CoinName'] = coin_name
    df['PredictedLabel'] = 0
    
    for i in tqdm(df.index):
        mat = np.zeros((1, DATA_POINTS_PER_WINDOW, DATA_POINTS_PER_WINDOW, encoded_feature_count), 'float32')    
        try:
            price = getMTFMatrix(df, 'Price', i)
            vol = getMTFMatrix(df, 'Volume', i)
        except:
            print("An exception occurred for coin when GADF encoded {} at {}".format(coin_name, i.strftime('%Y-%m-%d %H%M%S') ))
            continue
        
        mat[0][:,:,0] = price[0]
        mat[0][:,:,1] = vol[0]
        
        y_pred_R = np.round(mtfCnn3.predict(mat))
        df.loc[i, 'PredictedLabel'] = y_pred_R[0][0]
        
        if (df.loc[i, 'Label'] != y_pred_R[0][0]):
            mismatches.append( (i.strftime('%Y-%m-%d %H%M%S'), df.loc[i, 'Label'], y_pred_R[0][0]) )
            
    if  (len(mismatches) > 50 ):
        print ('******** Number of mismatches for coin{} is high={} !!!'.format(coin_name, len(mismatches)))
        print(mismatches)
    
    predictionDf = pd.concat([predictionDf, df[['CoinName', 'PredictedLabel', 'Label']]], axis=0)
    
    print('-------------- processed files %d' %fileIndex)
    print(psutil.virtual_memory())

In [ ]:
pred = predictionDf['PredictedLabel'].values
act = predictionDf['Label'].values

predictionDf.to_csv(<predictions_csv>, index=False)

In [ ]:
conf = confusion_matrix(predictionDf.Label, predictionDf.PredictedLabel)
print(conf)

clfr = classification_report(predictionDf.Label, predictionDf.PredictedLabel, output_dict=True)
print(clfr)

In [ ]:
import seaborn as sns
finConf=np.zeros((2,2), dtype=int)
for i in range(2):
    for j in range(2):
         finConf[i][j] += conf[i][j]
                
labels = ['True Neg','False Pos','False Neg','True Pos']
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(finConf/np.sum(finConf), annot=True, fmt='.2%', cmap='Blues')